In [2]:
import pykat

                                              ..-
    PyKat 0.8.6           _                  '(
                          \`.|\.__...-""""-_." )
       ..+-----.._        /  ' `            .-'
   . '            `:      7/* _/._\    \   (
  (        '::;;+;;:      `-"' =" /,`"" `) /
  L.        \`:::a:f            c_/     n_'
  ..`--...___`.  .    ,  
   `^-....____:   +.      www.gwoptics.org/pykat



# Pykat

This section aims to outline both the philosophy of Pykat and provide an overview of how it intends to work.

Pykat is an object-orientated Python wrapper for the Finesse binary. It aims to simplify the usage of Finesse for more complex simulation tasks. For example:
* Optimisation problems
* Chained simulations, feeding results of one simulation into the next
* Advanced plotting of outputs
* Automation of simulations
* Running simulations on clusters
* Using IPython/Jupyter notebooks to document simulation tasks

Interacting with Finesse to run more complicated simulation tasks using just the builtin plotting and output functionality quickly becomes limiting. In such cases it is usual to want to loop, repeat or modify simulations as you experiment and better understand the problem. Pykat has evolved to make such tasks easier.

There are three distinct features in the Pykat-Finesse interaction that will be referred to throughout this description:
* The **Model**: A particular optical setup or layout that 
* The **Simulation**: When a Finesse model is run to generate a particular output
* The **Output**: The data generated by a particular simulation

The **model** of the interferometer is that which you design and generate using Finesse commands. This will be a collection of optical components that represents the system in general. It does not assume anything about what data will be produced or detectors used.

Once a model is created you can design the type of **Simulation** that you wish to run. This requires you to first decide what type of output you want to see, thus what detectors you need to add to extract optical, mechanical or other values. Along with this the changing variable must also be set using the xaxis setting.

Finally, a simulation is taken and you **run** it. This takes all the settings and configuration of the simulations and passes it to the Finesse binary. This then runs the simulation and generates various data files containing the **output** result. Pykat parses these data files into more friendly formats for the user to enable plotting or further processing of the output.

<img style="width:90%;" src="images/pykat_outline.svg">

## Workflow - Creating the model and simulation

You can build your model and simulation with a kat object in two ways: By entering the standard Finesse commands as strings, loading a pre-written Finesse file, or by using the pykat objects.

### The kat object

Firstly, the most important object of pykat is the `kat` object. The name `kat` was chosen as this is the name of the Finesse binary. 

This object contains all the model details and represents one simulation that will be run. Thus, it is worthwhile to perceive this `kat` object as a single Finesse file that you might write by hand - except that this object handles the writing, running and processing of the output of the final file for you.

We create an empty model like so:

In [47]:
kat = pykat.finesse.kat()

In its current state this kat object doesn't do anything. You can load an existing Finesse file, if you have one, into the object like so:

In [48]:
kat.loadKatFile("cavity.kat")

Here a simple file consisting of a laser and a Fabry-Perot cavity has been added. You'll notice that no output is shown here. If there is an error in parsing any of the commands in a file these will show up at this stage.

This file contains a model of a Fabry-Perot cavity and the commands to simulate a scanning one of the mirrors to output the circulating power in it.

##### How do we see what is in the `kat` object at a particular moment?

For (nearly) every Finesse component, detector and command there is an equivalent Pykat object. For example, a laser is described by the object `pykat.components.pykat.components.laser`.

There are three main dictionaries that store the three different types of Finesse instructions: components, detectors and commands. The key to dictionary is the name of the component, detector or command and the value is the object itself. You can see the contents by printing them:

In [49]:
print(kat.components)
print(kat.detectors)
print(kat.commands)

{'l1': <pykat.components.pykat.components.laser_29 object at 0x10771b5c0>, 'm2': <pykat.components.pykat.components.mirror_32 object at 0x10775b048>, 'm1': <pykat.components.pykat.components.mirror_30 object at 0x1077585c0>, 's1': <pykat.components.pykat.components.space_31 object at 0x1067a0198>}
{'circ': <pykat.detectors.pykat.detectors.pd_3 object at 0x10775b7f0>}
{'xaxis': <pykat.commands.xaxis object at 0x10775b908>}


We can see here we have a laser called `l1`, mirrors `m1` and `m2`, a space `s1`, a single `pd` (DC photodiode) detector `circ` and an `xaxis` command.

**IMPORTANT NOTE: ** Do not add/remove/change the dictionary enteries yourself!! Doing this will break various internal connections and will just stop it working.

If you want to get hold of a particular object you can do it in two ways:

In [32]:
kat.components["m1"]

In [34]:
kat.m1

Note that the object reference, something like `0x10771bd30`, is the same in both cases. So we can see we are accessing the same object.

The second method is the recommended way when writing your scripts and know the name of the object, as this is less typing. It also allows for auto-complete to work on the object. So when typing code you can type `kat.m1.` then press the tab key to see all the possible parameters.

The first method though is still useful in cases where you need to access objects when you have the name in a string. A typical example might be looping over objects by their name:

In [42]:
for n in range(1,3):
    m = kat.components["m"+str(n)]
    print(repr(m)) # Here we print the object info

##### How do I interact with components/detectors/commands?

Once the objects have been added to the `kat` object we can change their parameters easily. For example, we can change the reflectivity and transmistivity of the input mirror with:

In [60]:
kat.m1.R = 0.6
kat.m1.T = 0.4

Likewise, we can quickly see what the value of particular parameters is:

In [61]:
print(kat.m1.R)

0.6


##### What is the final Finesse file that is run?

Remember that Pykat is just a set of tools that produces a complete Finesse file and then runs it. It is sometimes useful to be able to print the actual Finesse file that is run to double check you have the correct file or to better understand some result you were not expecting. To do this we have to generate all the Finesse commands first:

In [50]:
f = kat.generateKatScript()
print(f)

['% Generated by PyKat 29.02.2016 15:31:26\n', '\n', 'l l1 1.0 0.0 0.0 n0\n', 'm m1 0.5 0.5 0.0 n0 n1\n', 's s1 1.0 n1 n2\n', 'm m2 0.5 0.5 0.0 n2 n3\n', 'xaxis m2 phi lin 0 180 180\n', 'pd0 circ n1\n', 'yaxis abs\n']


As you can see, this returns a list where each element is an individual line of a Finesse file. This can be made more pretty with:

In [51]:
print("".join(f))

% Generated by PyKat 29.02.2016 15:31:26

l l1 1.0 0.0 0.0 n0
m m1 0.5 0.5 0.0 n0 n1
s s1 1.0 n1 n2
m m2 0.5 0.5 0.0 n2 n3
xaxis m2 phi lin 0 180 180
pd0 circ n1
yaxis abs



Here we can see all the components, detectors and commands for simulating the cavity scan to output the circulating power.

##### How do I run the simulation?

Running the simulation is straight forward:

In [53]:
out = kat.run()

--------------------------------------------------------------
Running kat - Started at 2016-02-29 15:32:07.058674

Finished in 0:00:00.018607


If all has worked then it will just print how long it took to run. The `out` object contains the simulation output.

In [ ]:
out.plot()

In [ ]:
### Finesse commands and files

To produce

### Multiple simulations on the same model

In [ ]:
model = pykat.finesse.kat()

kat1 = model.deepcopy()

kat2 = model.deepcopy()

### Using the object interface

## Further information